In [3]:
from bs4 import BeautifulSoup
import requests
import time
import datetime
import smtplib

In [35]:
from random import choice
# connect to website
url = 'https://data.un.org/en/iso/pk.html'
headers = { "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.0.0 Safari/537.36"}
page = requests.get(url,headers = headers)

soup = BeautifulSoup(page.text, 'html')


In [36]:
if page.status_code != 200:
    print(f"Error: Status code {page.status_code}")
    print(page.content)  # Print the raw content for debugging
else:
    soup1 = BeautifulSoup(page.content, "html.parser")


In [37]:
print(soup1)
soup2 = BeautifulSoup(soup1.prettify(), "html.parser")
print("-----------------------------------------------------------")
print(soup2)

<!DOCTYPE html>
<html lang="en"><head><meta charset="utf-8"/>
<title>UNData app</title>
<meta content="Mobile application includes 50+ statistical indicators for 229 countries/territories.
" name="UNData app"/>
<meta content="United Nations" name="author"/><meta content="width=device-width, initial-scale=1" name="viewport"/>
<script>(function(i,s,o,g,r,a,m){i['GoogleAnalyticsObject']=r;i[r]=i[r]||function(){(i[r].q=i[r].q||[]).push(arguments)},i[r].l=1*new Date();a=s.createElement(o),m=s.getElementsByTagName(o)[0];a.async=1;a.src=g;m.parentNode.insertBefore(a,m)})(window,document,'script','https://www.google-analytics.com/analytics.js','ga');ga('create', 'UA-3839409-1', 'auto');ga('send', 'pageview');</script>
<link href="../../common/css/pure.css" rel="stylesheet"/>
<link href="../../common/css/summary.css" rel="stylesheet"/>
<link href="../../common/css/popdiv.css" rel="stylesheet"/>
<link href="../../common/fixed-navigation-bar.css" rel="stylesheet"/>
<link href="../../common/css/co

As each table is under "`details`" tag. Which contains "`summary`" tag -> title and "`table`" tag-> actual data

In [38]:
#We use table wrappers
table_wrappers = soup.find_all("details")

In [39]:
print(table_wrappers)

[<details>
<summary>General Information</summary>
<table class="pure-table" style="width:100%"><tbody>
<tr class="pure-table"><td>Region </td>
<td> </td>
<td align="right">Southern Asia<sup></sup></td></tr>
<tr class="pure-table"><td>Population (000, 2021)</td>
<td> </td>
<td align="right">225 200<sup>a</sup></td></tr>
<tr class="pure-table"><td>Pop. density (per km2, 2021)</td>
<td> </td>
<td align="right">292.1<sup>a</sup></td></tr>
<tr class="pure-table"><td>Capital city </td>
<td> </td>
<td align="right">Islamabad<sup></sup></td></tr>
<tr class="pure-table"><td>Capital city pop. (000, 2021)</td>
<td> </td>
<td align="right">1 095.1<sup>b</sup></td></tr>
<tr class="pure-table"><td>UN membership date </td>
<td> </td>
<td align="right">30-Sep-47<sup></sup></td></tr>
<tr class="pure-table"><td>Surface area (km2)</td>
<td> </td>
<td align="right">796 095<sup>b</sup></td></tr>
<tr class="pure-table"><td>Sex ratio (m per 100 f)</td>
<td> </td>
<td align="right">106<sup>a</sup></td></tr>
<

In [51]:
import pandas as pd
title_list = []
#Loop through each table and extract title and data
for x, wrapper in enumerate(table_wrappers):
  # get title from summary
  titles = wrapper.find("summary")
  print(titles)
  table_title = titles.text.strip() if titles else f"Table_{x+1}"
  print(table_title)


  #extract table
  tables = wrapper.find_all("table")



  for y, table in enumerate(tables):
        rows = table.find_all("tr")
        table_data = []

         # Extract headers (if available)
        headers = [header.text.strip() for header in rows[0].find_all("th")] if rows[0].find_all("th") else None


        # Extract row data
        for row in rows:
            data = row.find_all("td")
            data = [cell.text.strip() for cell in data]
            if data:
                table_data.append(data)

          #create a data frame
        df = pd.DataFrame(table_data, columns=headers if headers else None)
        # Create a unique filename
        filename = f"{table_title.replace(' ', '_')}_Table_{y+1}.csv"

        # Save to CSV
        df.to_csv(filename, index=False)





<summary>General Information</summary>
General Information
<summary>Economic indicators</summary>
Economic indicators
<summary>Social indicators</summary>
Social indicators
<summary>Environment and infrastructure indicators</summary>
Environment and infrastructure indicators


In [46]:
print(df.head)

<bound method NDFrame.head of                                                     0                   1  \
0                                                                    2010   
1   Individuals using the Internet (per 100 inhabi...                  8h   
2    Research & Development expenditurep,q (% of GDP)                0.4r   
3                         Threatened species (number)                 109   
4                     Forested areah (% of land area)                 5.3   
5   CO2 emission estimates (million tons/tons per ...  129.2 /        0.7   
6             Energy production, primary (Petajoules)               2 040   
7               Energy supply per capita (Gigajoules)                  16   
8   Tourist/visitor arrivals at national borders (...                 907   
9   Important sites for terrestrial biodiversity p...                34.8   
10  Pop. using safely managed drinking water (urba...   45.7 /       32.3   
11  Net Official Development Assist. received 